In [ ]:
# Version 2


# Version 2

Some things I need to address in this version:

1. Currently the model has no understanding of empty nets
2. Currently, 5x4 / 4x5 refers to home and away, so the model doesn't know if the shot is coming from the powerplay or penalty kill
3. Should also include missed shots (go from shots to fenwick, can't do Corsi because blocked shots don't show shot origin)


4/26, Currently don't understand what a 5x3 is, but working on it!


Things to work on in the future

Create a better game prediction model that accounts for Rest, etc.

In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import seaborn as sns
import matplotlib.pyplot as plt

import data_prep

In [2]:
def create_input_data_from_csv(path_to_csv):
    df = pd.read_csv(path_to_csv)

    # get list of ilocs of rows where the 'Event' column is 'GOAL or SHOT or MISS'
    goal_shot_rows = df.loc[df['Event'].isin(['GOAL', 'SHOT', 'MISS'])]

    # shift the dataframe by 1 row
    shifted_df = df.shift(1)
    shifted_df.columns = ['prev_' + name for name in df.columns]

    # combine goal_shot_rows with shifted_df, but using rows from goal_shot_rows
    goal_shot_rows = pd.concat([goal_shot_rows, shifted_df.loc[goal_shot_rows.index]], axis=1)

    goal_shot_rows = goal_shot_rows[goal_shot_rows['Period'] != 0]
    goal_shot_rows = goal_shot_rows[goal_shot_rows['Period'] != 5]

    # Create "home_or_away" column based on if "Ev_team" is the same as "Home_Team" or "Away_Team"
    goal_shot_rows['home_or_away'] = goal_shot_rows.apply(lambda x: 'home' if x['Ev_Team'] == x['Home_Team'] else 'away', axis=1)
    
    # check to see if on power play if "home_or_away" is equal to "home" and "Strength" is equal to "5x4" or "home_or_away" is equal to "away" and "Strength" is equal to "4x5"
    goal_shot_rows['true_strength'] = goal_shot_rows.apply(lambda x: "5x4" if (x['home_or_away'] == 'home' and x['Strength'] == '5x4' ) or (x['home_or_away'] == 'away' and x['Strength'] == '4x5') else "5x5", axis=1)

    # similar as above, but for 5x3
    goal_shot_rows['true_strength'] = goal_shot_rows.apply(lambda x: "5x3" if (x['home_or_away'] == 'home' and x['Strength'] == '5x3' ) or (x['home_or_away'] == 'away' and x['Strength'] == '3x5') else x['true_strength'], axis=1)

    # similar as above, but for 6x5
    goal_shot_rows['true_strength'] = goal_shot_rows.apply(lambda x: "6x5" if (x['home_or_away'] == 'home' and x['Strength'] == '6x5' ) or (x['home_or_away'] == 'away' and x['Strength'] == '5x6') else x['true_strength'], axis=1)

    # similar as above, but for 4x3
    goal_shot_rows['true_strength'] = goal_shot_rows.apply(lambda x: "4x3" if (x['home_or_away'] == 'home' and x['Strength'] == '4x3' ) or (x['home_or_away'] == 'away' and x['Strength'] == '3x4') else x['true_strength'], axis=1)

    # similar as above,but for 3x3
    goal_shot_rows['true_strength'] = goal_shot_rows.apply(lambda x: "3x3" if (x['home_or_away'] == 'home' and x['Strength'] == '3x3' ) or (x['home_or_away'] == 'away' and x['Strength'] == '3x3') else x['true_strength'], axis=1)

    # check to see if a shot is on an empty net by checking if home_or_away is equal to "home" and "Away_Goalie" is equal to "None" or home_or_away is equal to "away" and "Home_Goalie" is equal to "None"
    goal_shot_rows['on_empty_net'] = goal_shot_rows.apply(lambda x: 1 if (x['home_or_away'] == 'home' and x['Away_Goalie'] == '') or (x['home_or_away'] == 'away' and x['Home_Goalie'] == '') else 0, axis=1)


    val_input_data = goal_shot_rows[['Event', 'Period', 'Seconds_Elapsed', 'Strength', 'Type', 'xC', 'yC', 'home_or_away', 'true_strength', 'on_empty_net', 'prev_Event', 'prev_Period', 'prev_Seconds_Elapsed', 'prev_Strength', 'prev_Type', 'prev_xC', 'prev_yC']]
    val_result_data = goal_shot_rows['Event']
    val_result_data = val_result_data.apply(lambda x: 1 if x == "GOAL" else 0)

    val_input_data = val_input_data.drop(columns=['Event'])

    # Split out the data between numeric values (can carry forward) and categorical values (need to be turned into binary columns)
    val_input_data_numeric = val_input_data[['Period', 'Seconds_Elapsed', 'xC', 'yC', 'on_empty_net','prev_Seconds_Elapsed', 'prev_xC', 'prev_yC']]
    val_input_data_categorical = pd.DataFrame()

    for column in ['Type', 'true_strength', 'home_or_away']:
        dummy_columns = pd.get_dummies(val_input_data[column])
        val_input_data_categorical = pd.concat([dummy_columns, val_input_data_categorical], axis=1)

    for column in ['prev_Event', 'prev_Strength', 'prev_Type']:
        dummy_columns = pd.get_dummies(val_input_data[column])

        # rename all columns with a prefix of "prev_"
        dummy_columns.columns = ['prev_' + str(col) for col in dummy_columns.columns]
        val_input_data_categorical = pd.concat([val_input_data_categorical, dummy_columns], axis=1)
        
    # change all True/False to 1/0
    val_input_data_categorical = val_input_data_categorical.applymap(lambda x: 1 if x == True else 0)

    # combine the two dataframes
    val_input_data_combined = pd.concat([val_input_data_numeric, val_input_data_categorical], axis=1)

    #set val_input_data2 to be all floats
    val_input_data_combined = val_input_data_combined.astype(float)

    columns = ['Seconds_Elapsed',                   'xC',                   'yC',
               'on_empty_net', 'prev_Seconds_Elapsed',              'prev_xC',
                    'prev_yC',                'is_PS',                 'away',
                       'home',                  '3x3',                  '4x3',
                        '5x3',                  '5x4',                  '5x5',
                        '6x5',             'BACKHAND',            'DEFLECTED',
                  'SLAP SHOT',            'SNAP SHOT',               'TIP-IN',
                'WRAP-AROUND',           'WRIST SHOT',                      1,
                            2,                      3,                      4,
                 'prev_BLOCK',             'prev_CHL',             'prev_FAC',
                  'prev_GEND',            'prev_GIVE',            'prev_GOAL',
                   'prev_HIT',            'prev_MISS',            'prev_PENL',
                  'prev_SHOT',            'prev_STOP',            'prev_TAKE',
                  'prev_-1x3',            'prev_-1x5',             'prev_0x1',
                   'prev_0x5',             'prev_3x3',             'prev_3x4',
                   'prev_3x5',             'prev_4x3',             'prev_4x4',
                   'prev_4x5',             'prev_5x0',             'prev_5x3',
                   'prev_5x4',             'prev_5x5',             'prev_5x6',
                   'prev_6x5',                'prev_',        'prev_BACKHAND',
             'prev_DEFLECTED',            'prev_PENL',       'prev_SLAP SHOT',
             'prev_SNAP SHOT',          'prev_TIP-IN',     'prev_WRAP-AROUND',
            'prev_WRIST SHOT']

    # For each column in input_data_combined, check if it exists in val_input_data_combined, if not add it as all 0s
    for column in columns:
        if column not in val_input_data_combined.columns:
                val_input_data_combined[column] = 0

    for column in val_input_data_combined.columns:
        if column not in columns:
                if column in val_input_data_combined.columns:
                    print(f"Column {column} not in columns")
                    val_input_data_combined = val_input_data_combined.drop(column, axis=1)


    # reduce val_input_data_combined to have the same columns as input_data_combined
    val_input_data_combined = val_input_data_combined[columns]

    # change all NaN numbers to 0
    val_input_data_combined = val_input_data_combined.fillna(0)

    print(val_input_data_combined.shape)

    return val_input_data_combined, val_result_data, goal_shot_rows

In [4]:
input_data, result_data, original_input = create_input_data_from_csv(r"C:\Users\JoshG\hockey_scraper_data\csvs\nhl_pbp_20182019.csv")

input_data2, result_data2, original_input2 = create_input_data_from_csv(r"C:\Users\JoshG\hockey_scraper_data\csvs\nhl_pbp_20202021.csv")
input_data3, result_data3, original_input3 = create_input_data_from_csv(r"C:\Users\JoshG\hockey_scraper_data\csvs\nhl_pbp_20222023.csv")

# combine the two dataframes
input_data = pd.concat([input_data, input_data2, input_data3])
result_data = pd.concat([result_data, result_data2, result_data3])
original_input = pd.concat([original_input, original_input2, original_input3])

C:\Users\JoshG\AppData\Local\Temp\ipykernel_41868\835714480.py:61: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  val_input_data_categorical = val_input_data_categorical.applymap(lambda x: 1 if x == True else 0)


Column Period not in columns
Column BLOCK not in columns
Column CHL not in columns
Column FAC not in columns
Column GEND not in columns
Column GIVE not in columns
Column GOAL not in columns
Column HIT not in columns
Column MISS not in columns
Column PENL not in columns
Column SHOT not in columns
Column STOP not in columns
Column TAKE not in columns
Column prev_DM #15 PETROVIC PS-Holding on breakaway(0 min) not in columns
Column prev_Fighting (maj) not in columns
Column prev_Game Misconduct(10 min) not in columns
Column prev_Hi-sticking(2 min) not in columns
Column prev_Holding(2 min) not in columns
Column prev_Hooking(2 min) not in columns
Column prev_Interference(2 min) not in columns
Column prev_Misconduct (10 min) not in columns
Column prev_PS-Holding on breakaway(0 min) not in columns
Column prev_PS-Hooking on breakaway(0 min) not in columns
Column prev_PS-Slash on breakaway(0 min) not in columns
Column prev_PS-Throw object at puck(0 min) not in columns
Column prev_PS-Tripping on b

C:\Users\JoshG\AppData\Local\Temp\ipykernel_41868\835714480.py:61: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  val_input_data_categorical = val_input_data_categorical.applymap(lambda x: 1 if x == True else 0)


Column Period not in columns
Column BLOCK not in columns
Column CHL not in columns
Column DELPEN not in columns
Column FAC not in columns
Column GEND not in columns
Column GIVE not in columns
Column HIT not in columns
Column MISS not in columns
Column PEND not in columns
Column PENL not in columns
Column SHOT not in columns
Column STOP not in columns
Column TAKE not in columns
Column prev_DELPEN not in columns
Column prev_PEND not in columns
Column prev_2x4 not in columns
Column prev_2x5 not in columns
Column prev_3x0 not in columns
Column prev_3x2 not in columns
Column prev_4x2 not in columns
Column prev_4x6 not in columns
Column prev_5x1 not in columns
Column prev_5x2 not in columns
Column prev_5x7 not in columns
Column prev_6x4 not in columns
Column prev_6x6 not in columns
Column prev_7x5 not in columns
Column prev_8x5 not in columns
Column prev_Hooking(2 min) not in columns
Column prev_PS-Covering puck in crease(0 min) not in columns
Column prev_PS-Holding on breakaway(0 min) not i

C:\Users\JoshG\AppData\Local\Temp\ipykernel_41868\835714480.py:61: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  val_input_data_categorical = val_input_data_categorical.applymap(lambda x: 1 if x == True else 0)


Column Period not in columns
Column BLOCK not in columns
Column CHL not in columns
Column DELPEN not in columns
Column FAC not in columns
Column GIVE not in columns
Column HIT not in columns
Column MISS not in columns
Column PENL not in columns
Column SHOT not in columns
Column STOP not in columns
Column TAKE not in columns
Column prev_DELPEN not in columns
Column prev_GY #8 TANEV PS-Hooking on breakaway(0 min) not in columns
Column prev_PS-Covering puck in crease(0 min) not in columns
Column prev_PS-Goalkeeper displaced net(0 min) not in columns
Column prev_PS-Holding on breakaway(0 min) not in columns
Column prev_PS-Hooking on breakaway(0 min) not in columns
Column prev_PS-Slash on breakaway(0 min) not in columns
Column prev_PS-Throw object at puck(0 min) not in columns
Column prev_PS-Tripping on breakaway(0 min) not in columns
(121992, 64)


In [5]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

x = torch.tensor(input_data.values, dtype=torch.float32)
y = torch.tensor(result_data.values, dtype=torch.float32)

x.to(device)
y.to(device)

model = nn.Sequential(
    nn.Linear(64, 180),
    nn.ReLU(),
    nn.Linear(180, 60),
    nn.ReLU(),
    nn.Linear(60, 1),
    nn.Sigmoid()
)

model.to(device)


loss_fn = nn.BCELoss()  # binary cross entropy
loss_fn.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)


Using device: cuda



In [20]:
n_epochs = 1000000
batch_size = 20000

print(len(y))
print(len(x))

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(n_epochs):
    accuracy_sum = 0
    for i in range(0, len(x), batch_size):
        Xbatch = x[i:i+batch_size]
        y_pred = model(Xbatch.cuda())
        ybatch = y[i:i+batch_size]
        #loss = loss_fn(y_pred, ybatch)
        loss = loss_fn(y_pred.squeeze(), ybatch.cuda())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # accuracy_sum += ((y_pred > 0.5).float() == ybatch.cuda()).float().mean()

    if (epoch % 1000 == 0):
        # accuracy = accuracy_sum / (len(y))
        print(f'{epoch}, {loss}')


# compute accuracy (no_grad is optional)
# y_pred = model(x.cuda()).cuda()
 

torch.save(model.state_dict(), '0429_xG_Model_1M.pt')

318330
318330
0, 0.18508656322956085
1000, 0.17786064743995667


In [11]:
torch.save(model.state_dict(), '0426Model_1M.pt')

In [7]:
filepath = '0426Model_100k.pt'

model.load_state_dict(torch.load(filepath))
model.eval()
model.cpu()



with torch.no_grad():
    y_pred = model(x.cpu())

# print dimensions of y_pred and y
print(y_pred.shape)
print(y.shape)

y_rounded = y_pred.round()
y_rounded.to('cpu')
y.to('cpu')

accuracy = (y_rounded.cpu() == y.cpu()).float().mean()

torch.save(model.state_dict(), '0426Model_100k.pt')

torch.Size([121992, 1])
torch.Size([121992])
